# Generate a Powerpoint Slide Deck based on Data in a Database

This is a simple example how a Powerpoint slide deck can be generated using Python. A typical use case is that an overview presentation of all projects needs to be created. The data about the projects is in a database, but instead of manually copying and pasting, the whole process can be automated.

For a good-looking presentation, a template has to be used.



## Step 0: Preparing the Environment

Install the libraries to manipulate Powerpoint files:

In [1]:
!pip install python-pptx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.9/159.9 kB 8.7 MB/s eta 0:00:00


Mount the Google Drive where the files are and set it as the working directory

In [2]:
from google.colab import drive

# Mount the Google Drive at mount
mount='/content/gdrive'
print("Colab: mounting Google drive on ", mount)

drive.mount(mount)

# Switch to the directory on the Google Drive that you want to use
import os
drive_root = mount + "/My Drive/Colab Notebooks/PPT Generation"


# Change to the directory
print("\nColab: Changing directory to ", drive_root)
%cd $drive_root

Colab: mounting Google drive on  /content/gdrive
Mounted at /content/gdrive

Colab: Changing directory to  /content/gdrive/My Drive/Colab Notebooks/PPT Generation
/content/gdrive/My Drive/Colab Notebooks/PPT Generation


## Step 1: Setting the parameters

First, let's define the name the Powerpoint template as well as the resulting Powerpoint slide deck. We also set the URL and get username and password from the secrets in Google Co-Labs (see left sidebar).

In [3]:
from google.colab import userdata

ppt_templatename = 'TwinCases_Template.pptx'
ppt_filename = 'TwinCases_Overview.pptx'

username = userdata.get("API_USERNAME")
password = userdata.get("API_PASSWORD")

url = 'https://shaller.pythonanywhere.com/usecases?scope=basic'



Now we call the API to get a Pandas data frame with the following columns:
* **name**: Name of the use case
* **description**: A short text describing the project
* **duration**: A text with from-to dates
* **illustrationLink**: A URL to a picture illustrating the project


In [4]:
import pandas as pd
import requests

column_names = ['name',
           'description',
           'duration',
           'illustrationLink']

def nstr(s):
  return '' if s is None else s

def def_image(img):
  return 'https://upload.wikimedia.org/wikipedia/commons/1/14/No_Image_Available.jpg' if img is None else img

def getUseCaseData(url, username, password):

  headers = {"Accept": "application/json"}

  session = requests.Session()
  session.auth = (username, password)

  # Make a GET request to the API
  response = session.get(url, headers=headers)

  # Check if the request was successful
  if response.status_code == 200:
      data = response.json()
      print("API response:", data)
      data_rows = []
      for uc in data:
        uc_data = uc['use_case']
        data_rows.append([uc_data['name'], uc_data['description'], f"{nstr(uc_data['startDate'])} - {nstr(uc_data['endDate'])}", def_image(uc_data['illustrationLink'] )])

      return pd.DataFrame(columns=column_names, data=data_rows)
  else:
      print(f"Error: {response.status_code} - {response.text}")
      return None



In [5]:
usecase_data = getUseCaseData(url, username, password)

usecase_data

API response: [{'use_case': {'id': 1, 'name': 'Urban Green Space Revitalization', 'description': 'This project focuses on transforming an underutilized urban area into a vibrant green space that includes community gardens, walking trails, and recreational areas. The goal is to enhance the quality of life for residents by providing a sustainable and accessible natural environment.', 'illustrationLink': 'https://www.acbconsultingservices.com/uploads/optimized/urban-green-space-projects-urban-greening-projects-46-a.jpg', 'startDate': '2022-12-01', 'endDate': '2013-11-30'}}, {'use_case': {'id': 2, 'name': 'Renewable Energy Initiative', 'description': 'This project aims to increase the adoption of renewable energy sources within a local community. It involves the installation of solar panels on residential and commercial buildings, as well as educational programs to promote energy efficiency and sustainability.', 'illustrationLink': 'https://www.moneyweb.co.za/wp-content/uploads/2015/10/shu

,name,description,duration,illustrationLink
0,Urban Green Space Revitalization,This project focuses on transforming an underu...,2022-12-01 - 2013-11-30,https://www.acbconsultingservices.com/uploads/...
1,Renewable Energy Initiative,This project aims to increase the adoption of ...,2024-02-01 -,https://www.moneyweb.co.za/wp-content/uploads/...
2,Historical Preservation Program,This project is dedicated to preserving and re...,March 2023 - March 2026,https://dcp.ufl.edu/historic-preservation/wp-c...
3,Mother Earth,"One of Europe’s newest weather satellites, the...",-,https://bgr.com/wp-content/uploads/2022/03/Ado...
4,Incognito,This project is secret,-,https://upload.wikimedia.org/wikipedia/commons...
5,Test,Show new test case,-,https://images-wixmp-ed30a86b8c4ca887773594c2....


## Step 2: Create the slide deck using the template

The current Python libary for manipulating Powerpoint slides doesn't support the deletion of slides. Hence, the template file shouldn't contain any slides!

Note: The following code doesn't include any error handling.

First we define a function that downloads an image:

In [6]:
from PIL import Image
from io import BytesIO

def download_image(url):

    # temporary file name
    file_name = 'temp_picture.jpg'

    # Send a GET request to the URL
    response = req.get(url)

    # Check if the request was successful
    if response.status_code == 200:
        # Open the image from the response content
        image = Image.open(BytesIO(response.content))
        image.save(file_name )
        return file_name
    else:
        print(f"Failed to retrieve image. HTTP Status code: {response.status_code}")
        return None

Now the rest of the code:


In [7]:
import pptx
import datetime as dt
import requests as req
from PIL import Image

# Open the template
presentation = pptx.Presentation(ppt_templatename)

# Clean up: Delete all existing slides  --- NOT SUPPORTED YET !!!!
# presentation.slides.clear()

# Get the layouts
layout_title = presentation.slide_master.slide_layouts.get_by_name('Title Slide')
layout_project_description = presentation.slide_master.slide_layouts.get_by_name('1_Comparison')


# Create title slide
title_slide = presentation.slides.add_slide(layout_title)

# As the library is not supporting the look up of shapes by name, we define the indexes here
shape_title = 0
shape_subtitle = 1

#Insert title slide text
title_slide.shapes[shape_title].text = 'Use Cases for Urban Digital Twins'
title_slide.shapes[shape_subtitle].text = dt.datetime.today().strftime('%B %Y')

# Iterate through data and create one slide per project
for idx in usecase_data.index:
  print(usecase_data['name'][idx], ' ', usecase_data['illustrationLink'][idx])
  # Create project slide
  project_slide = presentation.slides.add_slide(layout_project_description)

  # As the library is not supporting the look up of shapes by name, we define the indexes here
  prj_name = 0
  prj_desc = 1
  prj_duration = 2
  prj_image = 3


  #Insert title slide text
  project_slide.shapes[prj_name].text = usecase_data['name'][idx]
  project_slide.shapes[prj_desc].text = usecase_data['description'][idx]
  project_slide.shapes[prj_duration].text = usecase_data['duration'][idx]
  illustration = project_slide.shapes[prj_image]
  try:
    illustration.insert_picture(download_image(usecase_data['illustrationLink'][idx]))
  except:
    illustration.insert_picture(download_image('https://storage.needpix.com/rsynced_images/fail-test.jpg'))

# Save and close presentation
presentation.save(ppt_filename)

Urban Green Space Revitalization   https://www.acbconsultingservices.com/uploads/optimized/urban-green-space-projects-urban-greening-projects-46-a.jpg
Renewable Energy Initiative   https://www.moneyweb.co.za/wp-content/uploads/2015/10/shutterstock_201614372.jpg
Failed to retrieve image. HTTP Status code: 403
Historical Preservation Program   https://dcp.ufl.edu/historic-preservation/wp-content/uploads/sites/14/2021/08/IMG_6936-1-2048x1536.jpg

Failed to retrieve image. HTTP Status code: 404
Mother Earth   https://bgr.com/wp-content/uploads/2022/03/AdobeStock_194080021.jpeg?resize=1020%2C574&quality=82
Incognito   https://upload.wikimedia.org/wikipedia/commons/1/14/No_Image_Available.jpg
Test   https://images-wixmp-ed30a86b8c4ca887773594c2.wixmp.com/f/715425db-6c68-4fe6-9ed9-bc71814092e1/dexf5py-1a5bb40e-2e09-454a-80d2-a972a74d7741.jpg?token=eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJzdWIiOiJ1cm46YXBwOjdlMGQxODg5ODIyNjQzNzNhNWYwZDQxNWVhMGQyNmUwIiwiaXNzIjoidXJuOmFwcDo3ZTBkMTg4OTgyMjY0MzczYT